In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# BigQuery
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project="project='TC-DA-1")




In [3]:
# Construct a reference to the "Adventure work 2005" dataset
dataset_ref = client.dataset("adwentureworks_db", project="tc-da-1")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [4]:
def query_to_df(query):
    # transfers query results to pandas dataframe for easy manipulating
    return(client.query(query, project="tc-da-1").result().to_dataframe())

def get_schema(table):
    # retreives the schema as a printed object
    return(client.get_table(table).schema)

In [5]:
# List all the tables in the  dataset
tables = list(client.list_tables(dataset_ref))
print(f"There are {len(tables)} tables in the dataset")
# Print names of all tables in the dataset 
for table in tables:  
    print(table.table_id)


There are 65 tables in the dataset
EmployeeDepartmentHistory
address
addresstype
billofmaterials
contact
contactcreditcard
contacttype
countryregion
countryregioncurrency
creditcard
culture
currency
currencyrate
customer
customeraddress
department
document
employee
employeeaddress
employeedepartmenthistory
employeepayhistory
individual
location
product
productcategory
productcosthistory
productdescription
productdocument
productinventory
productlistpricehistory
productmodel
productmodelillustration
productmodelproductdescriptionculture
productphoto
productproductphoto
productreview
productsubcategory
productvendor
purchaseorderdetail
purchaseorderheader
salesorderdetail
salesorderheader
salesorderheadersalesreason
salesperson
salespersonquotahistory
salesreason
salestaxrate
salesterritory
salesterritoryhistory
scrapreason
shift
shipmethod
shoppingcartitem
specialoffer
specialofferproduct
stateprovince
storecontact
transactionhistory
transactionhistoryarchive
unitmeasure
vendor
vendorad

In [6]:
for table in tables:
    print(f"{table.table_id}: {get_schema(f'tc-da-1.adwentureworks_db.{table.table_id}')}")

EmployeeDepartmentHistory: [SchemaField('EmployeeID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('DepartmentID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('ShiftID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('StartDate', 'DATETIME', 'NULLABLE', None, (), None), SchemaField('EndDate', 'DATETIME', 'NULLABLE', None, (), None), SchemaField('ModifiedDate', 'TIMESTAMP', 'NULLABLE', None, (), None)]
address: [SchemaField('AddressID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('AddressLine1', 'STRING', 'NULLABLE', None, (), None), SchemaField('AddressLine2', 'STRING', 'NULLABLE', None, (), None), SchemaField('City', 'STRING', 'NULLABLE', None, (), None), SchemaField('StateProvinceID', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('PostalCode', 'STRING', 'NULLABLE', None, (), None), SchemaField('rowguid', 'BYTES', 'NULLABLE', None, (), None), SchemaField('ModifiedDate', 'TIMESTAMP', 'NULLABLE', None, (), None)]
addresstype: [SchemaField('AddressTypeID

In [7]:
 QUERY = (
     """
     SELECT p.ProductId, p.Name, p.ProductNumber, p.size, p.color, p.ProductSubcategoryId, s.Name AS SubcategoryName
     FROM `tc-da-1.adwentureworks_db.product`AS p
     INNER JOIN `tc-da-1.adwentureworks_db.productsubcategory` AS s 
     ON p.ProductSubcategoryId=s.ProductSubcategoryId
     ORDER BY s.Name 
     """
 )
    
query_to_df(QUERY)

,ProductId,Name,ProductNumber,size,color,ProductSubcategoryId,SubcategoryName
0,855,"Men's Bib-Shorts, S",SB-M891-S,S,Multi,18,Bib-Shorts
1,857,"Men's Bib-Shorts, L",SB-M891-L,L,Multi,18,Bib-Shorts
2,856,"Men's Bib-Shorts, M",SB-M891-M,M,Multi,18,Bib-Shorts
3,876,Hitch Rack - 4-Bike,RA-H123,None,None,26,Bike Racks
4,879,All-Purpose Bike Stand,ST-1401,None,None,27,Bike Stands
...,...,...,...,...,...,...,...
290,828,HL Road Rear Wheel,RW-R820,None,Black,17,Wheels
291,823,LL Mountain Rear Wheel,RW-M423,None,Black,17,Wheels
292,824,ML Mountain Rear Wheel,RW-M762,None,Black,17,Wheels
293,825,HL Mountain Rear Wheel,RW-M928,None,Black,17,Wheels


In [8]:
QUERY = (
     """
    SELECT p.ProductId, p.Name, p.ProductNumber, p.size, p.color, p.ProductSubcategoryId, s.Name AS SubCategory, c.Name AS Category
     FROM ((`tc-da-1.adwentureworks_db.product`AS p
     INNER JOIN `tc-da-1.adwentureworks_db.productsubcategory` AS s
     ON p.ProductSubcategoryId=s.ProductSubcategoryId)
    INNER JOIN `tc-da-1.adwentureworks_db.productcategory` AS c
   ON s.ProductCategoryId=c.ProductCategoryId)
     ORDER BY c.Name
     """
 )
    
query_to_df(QUERY)

,ProductId,Name,ProductNumber,size,color,ProductSubcategoryId,SubCategory,Category
0,711,"Sport-100 Helmet, Blue",HL-U509-B,None,Blue,31,Helmets,Accessories
1,707,"Sport-100 Helmet, Red",HL-U509-R,None,Red,31,Helmets,Accessories
2,708,"Sport-100 Helmet, Black",HL-U509,None,Black,31,Helmets,Accessories
3,880,Hydration Pack - 70 oz.,HY-1023-70,70,Silver,32,Hydration Packs,Accessories
4,846,Taillights - Battery-Powered,LT-T990,None,None,33,Lights,Accessories
...,...,...,...,...,...,...,...,...
290,824,ML Mountain Rear Wheel,RW-M762,None,Black,17,Wheels,Components
291,825,HL Mountain Rear Wheel,RW-M928,None,Black,17,Wheels,Components
292,826,LL Road Rear Wheel,RW-R623,None,Black,17,Wheels,Components
293,894,Rear Derailleur,RD-2308,None,Silver,9,Derailleurs,Components


In [9]:
     
      QUERY = (
     """
   SELECT p.ProductId, p.Name, p.ProductNumber, p.size, p.color, p.ProductSubcategoryId, s.Name AS SubCategory, c.Name AS Category, p.ListPrice AS Price
     FROM ((`tc-da-1.adwentureworks_db.product`AS p
     INNER JOIN `tc-da-1.adwentureworks_db.productsubcategory` AS s
     ON p.ProductSubcategoryId=s.ProductSubcategoryId)
    INNER JOIN `tc-da-1.adwentureworks_db.productcategory` AS c
   ON s.ProductCategoryId=c.ProductCategoryId)
   WHERE p.ListPrice>2000 AND p.SellEndDate IS NULL
     ORDER BY (SELECT p.ListPrice FROM `tc-da-1.adwentureworks_db.productcategory`  WHERE c.Name = "Bike")
    
     """
 )
    
query_to_df(QUERY)

,ProductId,Name,ProductNumber,size,color,ProductSubcategoryId,SubCategory,Category,Price
0,796,"Road-250 Black, 58",BK-R89B-58,58,Black,2,Road Bikes,Bikes,2443.35
1,795,"Road-250 Black, 52",BK-R89B-52,52,Black,2,Road Bikes,Bikes,2443.35
2,792,"Road-250 Red, 58",BK-R89R-58,58,Red,2,Road Bikes,Bikes,2443.35
3,794,"Road-250 Black, 48",BK-R89B-48,48,Black,2,Road Bikes,Bikes,2443.35
4,793,"Road-250 Black, 44",BK-R89B-44,44,Black,2,Road Bikes,Bikes,2443.35
5,956,"Touring-1000 Yellow, 54",BK-T79Y-54,54,Yellow,3,Touring Bikes,Bikes,2384.07
6,957,"Touring-1000 Yellow, 60",BK-T79Y-60,60,Yellow,3,Touring Bikes,Bikes,2384.07
7,968,"Touring-1000 Blue, 54",BK-T79U-54,54,Blue,3,Touring Bikes,Bikes,2384.07
8,967,"Touring-1000 Blue, 50",BK-T79U-50,50,Blue,3,Touring Bikes,Bikes,2384.07
9,955,"Touring-1000 Yellow, 50",BK-T79Y-50,50,Yellow,3,Touring Bikes,Bikes,2384.07


In [10]:
     
      QUERY = (
     """
   SELECT LocationID, COUNT( DISTINCT WorkOrderID) AS no_of_workorder,  COUNT( DISTINCT ProductID) as no_of_product, SUM(ActualCost) AS total_actual_cost, 
   
     FROM `tc-da-1.adwentureworks_db.workorderrouting` 
     WHERE ActualStartDate BETWEEN '2004-01-01'AND '2004-01-31'
     GROUP BY  LocationID
   
     """
 )
    
query_to_df(QUERY)

,LocationID,no_of_workorder,no_of_product,total_actual_cost
0,10,409,18,37730.25
1,20,284,14,24850.00
2,30,79,6,1145.50
3,40,48,3,1512.00
4,45,31,3,1116.00
5,50,1304,48,47922.00
6,60,957,77,46893.00
